<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/cloth_Baraff_Witkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install meshzoo

In [ ]:
from collections import namedtuple
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import meshzoo
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rc('animation', html='jshtml')

In [ ]:
vertex_positions_uv, triangle_vertex_indices = meshzoo.rectangle_tri(
    (0.0, 0.0),
    (1.0, 1.0),
    n=5,
    variant="zigzag",  # or "up", "down", "center"
)

print(vertex_positions_uv.shape)
print(triangle_vertex_indices.shape)

In [ ]:
n_vertices = vertex_positions_uv.shape[0]
vertex_positions_z = np.zeros(n_vertices)
vertex_positions = np.column_stack([vertex_positions_uv, vertex_positions_z]).flatten()

In [ ]:
def plot_cloth(ax, vertex_positions, triangle_vertex_indices):
    # x, y, z = vertex_positions.transpose()
    x = vertex_positions[0::3]
    y = vertex_positions[1::3]
    z = vertex_positions[2::3]

    ax.clear()  # necessary for the animations
    ax.plot_trisurf(x, y, z, triangles=triangle_vertex_indices, color='deepskyblue')
    ax.scatter(x, y, z, c='deeppink', s=20, depthshade=False)
    for i, (xi, yi, zi) in enumerate(zip(x, y, z)):
        ax.text(xi, yi, zi, str(i), fontsize='medium', color='black', zorder=10)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_xlim([-0.1, 1.1])
    ax.set_ylim([-0.1, 1.1])
    ax.set_zlim([-2, 0.25])

fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')
plot_cloth(ax, vertex_positions, triangle_vertex_indices)

In [ ]:
def triangle_area(triangle_vertices):
    v0, v1, v2 = triangle_vertices
    return np.linalg.norm(np.cross(v1 - v0, v2 - v0)) / 2.0

def calculate_area(triangles_indices, vertex_positions):
    area = 0.0
    for triangle in triangle_vertex_indices:
        triangle_vertices = vertex_positions[triangle]
        area += triangle_area(triangle_vertices)
    return area

def calculate_vertex_masses(vertex_positions_uv, triangle_vertex_indices, density=0.2):
    vertex_masses = np.zeros(vertex_positions_uv.shape[0])

    for i,j,k in triangle_vertex_indices:
        area = triangle_area(vertex_positions_uv[[i, j, k]])
        triangle_mass = density * area
        vertex_masses[i] += 1/3 * triangle_mass 
        vertex_masses[j] += 1/3 * triangle_mass 
        vertex_masses[k] += 1/3 * triangle_mass

    return vertex_masses

def calculate_triangle_constants(triangle_vertex_positions_uv):
    uv0, uv1, uv2 = triangle_vertex_positions_uv
    u0, v0 = uv0
    u1, v1 = uv1
    u2, v2 = uv2

    delta_u1 = u1 - u0
    delta_u2 = u2 - u0
    delta_v1 = v1 - v0
    delta_v2 = v2 - v0

    delta_u_matrix = np.array([(delta_u1, delta_u2),
                               (delta_v1, delta_v2)])
    
    inverted_delta_u_matrix = np.linalg.inv(delta_u_matrix)

    dw_denominator = (delta_u1 * delta_v2 - delta_u2 * delta_v1)

    dwu_dx = {}
    dwu_dx[0] = (delta_v1 - delta_v2) / dw_denominator
    dwu_dx[1] = delta_v2 / dw_denominator
    dwu_dx[2] = -delta_v1 / dw_denominator

    dwv_dx = {}
    dwv_dx[0] = (delta_u2 - delta_u1) / dw_denominator
    dwv_dx[1] = -delta_u2 / dw_denominator
    dwv_dx[2] = delta_u1 / dw_denominator

    return dwu_dx, dwv_dx, inverted_delta_u_matrix

In [ ]:
# Required attributes for a Baraff-Witkin style cloth simulator
class ClothSimulationMesh:
    def __init__(self, vertex_positions, vertex_positions_uv, triangle_vertex_indices):
        self.n_vertices = vertex_positions_uv.shape[0]
        self.n_triangles = triangle_vertex_indices.shape[0]

        # State 
        self.vertex_positions = vertex_positions.flatten().copy()
        self.vertex_velocities = np.zeros_like(self.vertex_positions)  

        # Configuration / Parameters
        self.triangle_vertex_indices = triangle_vertex_indices
        self.vertex_positions_uv = vertex_positions_uv
        self.vertex_masses = calculate_vertex_masses(vertex_positions_uv, triangle_vertex_indices)  
        self.triangle_stretch_stiffness_u = 1000.0 * np.ones(self.n_triangles) 
        self.triangle_stretch_stiffness_v = 1000.0 * np.ones(self.n_triangles)
        self.pinned_vertices = [] # np.zeros(self.n_vertices) could also be a boolean attribute too

        # Simulation (intermediate/cache) storage
        self.vertex_forces = np.zeros_like(self.vertex_velocities)
        self.vertex_forces_derivatives = np.zeros((3 * n_vertices, 3* n_vertices))

        # Precomputed quantities
        self.triangle_areas_uv = [triangle_area(vertex_positions_uv[tri]) for tri in triangle_vertex_indices]

        self.triangle_inverted_delta_u_matrices = []
        self.triangle_wu_derivatives = []
        self.triangle_wv_derivatives = []
        self.precompute_triangle_constants()


    def precompute_triangle_constants(self):
        for triangle_index in range(self.n_triangles):
            triangle = triangle_vertex_indices[triangle_index]
            dwu_dx, dwv_dx, inverted_delta_u_matrix = calculate_triangle_constants(vertex_positions_uv[triangle])
            
            self.triangle_wu_derivatives.append(dwu_dx)
            self.triangle_wv_derivatives.append(dwv_dx)
            self.triangle_inverted_delta_u_matrices.append(inverted_delta_u_matrix)

    def reset_vertex_forces_and_derivatives(self):
        self.vertex_forces.fill(0)
        self.vertex_forces_derivatives.fill(0)

    def dw_dx(self, triangle_index):
        dwu_dx = self.triangle_wu_derivatives[triangle_index]
        dwv_dx = self.triangle_wv_derivatives[triangle_index]
        return dwu_dx, dwv_dx

    def k_stretch(self, triangle_index):
        ku = self.triangle_stretch_stiffness_u[triangle_index]
        kv = self.triangle_stretch_stiffness_v[triangle_index]
        return ku, kv

cloth = ClothSimulationMesh(vertex_positions, vertex_positions_uv, triangle_vertex_indices)

In [ ]:
def slice3(i):
    return slice(3*i, 3*i+3)

In [ ]:
class DeformationGradient():
    """
    The derivatives wu and wv in the BW98 paper are generally known as the 
    deformation gradient in continuum and deformation mechanics.

    B&W describe w as "a map from plane coordinates to world space." (I assume
    they chose the letter w for "world"). Remember that the plane coordinates
    represent the cloth at rest. This means that w is a measure of the
    deformation between the rest pose of the cloth and the deformed world pose. 

    Besides the derivatives, the class also stores their length and normalized 
    forms for convenience, because they are needed several times later.
    """
    def __init__(self, cloth, triangle_index):
        i, j, k = cloth.triangle_vertex_indices[triangle_index]
        x0 = cloth.vertex_positions[slice3(i)]
        x1 = cloth.vertex_positions[slice3(j)]
        x2 = cloth.vertex_positions[slice3(k)]

        delta_x1 = x1 - x0
        delta_x2 = x2 - x0

        inverted_delta_u_matrix = cloth.triangle_inverted_delta_u_matrices[triangle_index]
            
        delta_x_matrix = np.column_stack((delta_x1, delta_x2))

        # Equation (9) in Baraff-Witkin.
        w_uv = delta_x_matrix @ inverted_delta_u_matrix
        
        wu, wv = np.hsplit(w_uv, 2)
        wu_norm = np.linalg.norm(wu)
        wv_norm = np.linalg.norm(wv)

        self.u = wu
        self.v = wv
        self.u_norm = wu_norm 
        self.v_norm = wv_norm
        self.u_normalized = wu / wu_norm
        self.v_normalized = wv / wv_norm



# Code to verify the derivatives, in production this could be a nice test
epsilon = 0.001
for triangle_index in range(cloth.n_triangles):
    print('=== Triangle ', triangle_index, ' ===========================')

    for i, v in enumerate(cloth.triangle_vertex_indices[triangle_index]):
        w0 = DeformationGradient(cloth, triangle_index)
        cloth.vertex_positions[slice3(v)][0] += epsilon # x
        w1 = DeformationGradient(cloth, triangle_index)
        cloth.vertex_positions[slice3(v)][1] += epsilon # y
        w2 = DeformationGradient(cloth, triangle_index)
        cloth.vertex_positions[slice3(v)][2] += epsilon # z
        w3 = DeformationGradient(cloth, triangle_index)

        dwu_dx0_x = (w1.u - w0.u) / epsilon
        dwu_dx0_y = (w2.u - w1.u) / epsilon
        dwu_dx0_z = (w3.u - w2.u) / epsilon

        print('Derivatives for vertex ', v, ' (=' + str(i) + '):')
        print(dwu_dx0_x.flatten())
        print(dwu_dx0_y.flatten())
        print(dwu_dx0_z.flatten())
        print(cloth.triangle_wu_derivatives[triangle_index][i])

In [ ]:
def calculate_strech_force(cloth, triangle_index, w):
    vertices = cloth.triangle_vertex_indices[triangle_index]
    area_uv = cloth.triangle_areas_uv[triangle_index]
    dwu_dx, dwv_dx = cloth.dw_dx(triangle_index)

    Cu = area_uv * (w.u_norm - 1.0)
    Cv = area_uv * (w.v_norm - 1.0)

    ku, kv = cloth.k_stretch(triangle_index)
    dCu_dx = {}
    dCv_dx = {}

    # Forces
    for m in range(3):
        dCu_dx[m] = area_uv * dwu_dx[m] * w.u_normalized
        dCv_dx[m] = area_uv * dwv_dx[m] * w.v_normalized

        # Equation (7) in Baraff-Witkin.
        force_m = -ku * Cu * dCu_dx[m] 
        force_m += -kv * Cv * dCv_dx[m]
        cloth.vertex_forces[slice3(vertices[m])] += force_m.reshape((3,))


    # Force derivatives        
    I_wu_wuT = np.identity(3) - np.outer(w.u_normalized, w.u_normalized)
    I_wv_wvT = np.identity(3) - np.outer(w.v_normalized, w.v_normalized)

    for m in range(3):
        for n in range(3):
            dCu_dx_mn = area_uv / w.u_norm * dwu_dx[m] * dwu_dx[n] * I_wu_wuT
            dfu_dx_mn = -ku * (np.outer(dCu_dx[m], dCu_dx[n]) + dCu_dx_mn * Cu)

            dCv_dx_mn = area_uv / w.v_norm * dwv_dx[m] * dwv_dx[n] * I_wv_wvT
            dfv_dx_mn = -kv * (np.outer(dCv_dx[m], dCv_dx[n]) + dCv_dx_mn * Cv)

            cloth.vertex_forces_derivatives[slice3(vertices[m]), slice3(vertices[n])] += dfu_dx_mn + dfu_dx_mn

def calculate_forces(cloth):
    for triangle_index in range(cloth.n_triangles):
        # Technical note: the w here is not the w(u, v) function from the paper, 
        # w is container for the derivatives wu and wv of w.
        w = DeformationGradient(cloth, triangle_index)
        calculate_strech_force(cloth, triangle_index, w)

In [ ]:
def step_forward_euler(cloth, dt):
    cloth.reset_vertex_forces_and_derivatives()
    calculate_forces(cloth)

    vertex_accelerations = np.zeros_like(cloth.vertex_velocities)
    vertex_accelerations[2::3] -= 9.81 # acceleration due to gravity

    M_inv = np.identity(3 * cloth.n_vertices) / np.repeat(cloth.vertex_masses, 3)
    vertex_accelerations += M_inv @ cloth.vertex_forces

    # print('total force on vertex 1: ', np.array(cloth.vertex_forces[slice3(1)]))
    # print()

    for i in cloth.pinned_vertices:
        vertex_accelerations[slice3(i)] = 0.0

    # integration
    cloth.vertex_velocities += vertex_accelerations * dt
    cloth.vertex_positions += cloth.vertex_velocities * dt


def step_backward_euler(cloth, dt):
    cloth.reset_vertex_forces_and_derivatives()
    calculate_forces(cloth)

    vertex_accelerations = np.zeros_like(cloth.vertex_velocities)

    vertex_accelerations[2::3] -= 9.81 # acceleration due to gravity

    M_inv = np.identity(3 * cloth.n_vertices) / np.repeat(cloth.vertex_masses, 3)

    # vertex_accelerations = M-1 @ f0 in the paper
    vertex_accelerations += M_inv @ cloth.vertex_forces

    # vertex_accelerations[cloth.pinned_vertices] = 0.0

    dfdx_v0 = cloth.vertex_forces_derivatives @ cloth.vertex_velocities
    M_h_dfdx_v0 = M_inv @ (dt * dfdx_v0)

    b = dt * (vertex_accelerations + M_h_dfdx_v0)

    A = np.identity(3 * cloth.n_vertices)
    A -= dt * M_inv @ cloth.vertex_forces_derivatives

    delta_v = x = np.linalg.solve(A, b)
    # print('residual = ', np.sum(A @ x - b))

    # integration
    cloth.vertex_velocities += delta_v

    for i in cloth.pinned_vertices:
        cloth.vertex_velocities[slice3(i)] = 0.0

    cloth.vertex_positions += cloth.vertex_velocities * dt


cloth = ClothSimulationMesh(vertex_positions, vertex_positions_uv, triangle_vertex_indices)
step_backward_euler(cloth, 0.001)

In [ ]:
# timesteps = 2000  
# dt = 0.001
timesteps = 20000  
dt = 0.0001

cloth = ClothSimulationMesh(vertex_positions, vertex_positions_uv, triangle_vertex_indices)
cloth.pinned_vertices = [0, 4]

def simulate(cloth, timesteps, dt):
    history = [cloth.vertex_positions.copy()]
    for _ in range(timesteps):
        step_backward_euler(cloth, dt)
        history.append(cloth.vertex_positions.copy())

    return history

history = simulate(cloth, timesteps, dt)

In [ ]:
def animate_cloth(history, dt, fps=50):
    fig = plt.figure(figsize=(5, 5), dpi=100)
    fig.subplots_adjust(0,0,1,1,0,0) # less padding
    ax = fig.add_subplot(111, projection='3d')
    plt.close()  # prevents duplicate output 

    fps_simulation = 1 / dt
    skip = np.floor(fps_simulation / fps).astype(np.int32)
    fps_adjusted = fps_simulation / skip
    print('fps was adjusted to:', fps_adjusted)

    def animate(i):
        j = min(i * skip, len(history) - 1)
        plot_cloth(ax, history[j], triangle_vertex_indices)
        ax.text2D(0.1, 0.9, 't = {:.3f}s'.format(j * dt), transform=ax.transAxes)


    n_frames = (len(history) - 1) // skip + 1
    interval = 1000*dt*skip
    anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=interval)
    return anim

animate_cloth(history, dt)